<p align="center" width="100%">
    <img width="40%" src="customer_support_icon.JPG"> 
</p>

A retail company is on a transformative journey, aiming to elevate their customer services through cutting-edge advancements in Speech Recognition and Natural Language Processing (NLP). As the machine learning engineer for this initiative, you are tasked with developing functionalities that not only convert customer support audio calls into text but also explore methodologies to extract insights from transcribed texts.

In this dynamic project, we leverage the power of `SpeechRecognition`, `Pydub`, and `spaCy` – three open-source packages that form the backbone of your solution. Your objectives are:
  - Transcribe a sample customer audio call, stored at `sample_customer_call.wav`, to showcase the power of open-source speech recognition technology.
  - Analyze sentiment, identify common named entities, and enhance user experience by searching for the most similar customer calls based on a given query from a subset of their pre-transcribed call data, stored at `customer_call_transcriptions.csv`.

This project is an opportunity to unlock the potential of machine learning to revolutionize customer support. Let's delve into the interplay between technology and service excellence.

In [105]:
!pip install SpeechRecognition
!pip install pydub
!pip install spacy
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download en_core_web_md

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Apps > Advanced app settings > App execution aliases.
Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Apps > Advanced app settings > App execution aliases.


In [106]:
# Import required libraries
import nltk
import spacy
import pandas as pd
import numpy as np
import speech_recognition as sr

# nltk.download('vader_lexicon')

from pydub import AudioSegment
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix


In [107]:
# Is the audio compatible for future speech recognition modeling?

recognizer = sr.Recognizer()
customer_call = sr.AudioFile("sample_customer_call.wav")

with customer_call as source:
    customer_audio = recognizer.record(source)

transcribed_text = recognizer.recognize_google(customer_audio)

customer_call = AudioSegment.from_file("sample_customer_call.wav")

frame_rate = customer_call.frame_rate
number_channels = customer_call.channels

In [108]:
# How many calls have a true positive sentiment?

df = pd.read_csv("customer_call_transcriptions.csv")
df.head()

X = df["text"]
y = df["sentiment_label"]

sid = SentimentIntensityAnalyzer()

def get_sentiment(x):
    compound_score = sid.polarity_scores(x)["compound"]
    if(compound_score >= 0.05):
        return "positive"
    elif(compound_score <= -0.05):
        return "negative"
    return "neutral"

y_pred = []

for x in X:
    y_pred.append(get_sentiment(x))

true_positive = 0
for i in range(len(y)):
    if(y[i] == "positive" and y_pred[i] == "positive"):
        true_positive += 1
true_positive

2

In [109]:
print(y_pred[3] == "positive")

False


In [110]:
# What is the most frequently named entity across all of the transcriptions?

nlp = spacy.load("en_core_web_sm")

entity_count = {}

for text in df["text"]:
    doc = nlp(text)
    for ent in doc.ents:
        named_enity = ent.text
        if named_enity not in entity_count:
            entity_count[named_enity] = 0
        else:
            entity_count[named_enity] += 1

max_freq = max(entity_count.values())
most_freq_ent = [key for key in entity_count if entity_count[key] == max_freq][0]
most_freq_ent

'yesterday'

In [111]:
max_freq

14

In [112]:
# Which call is the most similar to "wrong package delivery"?

nlp = spacy.load("en_core_web_md")
keyword = nlp("wrong package delivery")
similarity_score = []

for text in df["text"]:
    doc = nlp(text)
    score = doc.similarity(keyword)
    similarity_score.append(score)

max_index = similarity_score.index(max(similarity_score))
most_similar_text = df["text"][max_index]
most_similar_text

'wrong package delivered'

In [113]:
print(frame_rate)
print(number_channels)
print(true_positive)
print(most_freq_ent)
print(most_similar_text)

44100
1
2
yesterday
wrong package delivered
